
<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con la demanda eléctrica.** Las provincias con un PIB más alto o en crecimiento experimentan una mayor demanda de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Extracción

En el laboratorio de hoy tendrás que extraer la información necesaria para obtener tu objetivo de las siguientes fuentes de datos (deberás usar API's y herramientas de *web scrapping*):

- **Datos de la API de Red Eléctrica Española (REE):** Deberás extraer datos mensuales a nivel provincial de los siguientes aspectos:

  - **Demanda Eléctrica:** Nos proporciona los datos de demanda eléctrica a nivel provincial, agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/demanda/evolucion", añadiendo los parámetros que sean necesarios. 

  - **Generación Eléctrica:** Nos proporciona los datos de generación eléctrica a nivel provincial, diferenciando entre fuentes de energía (eólica, solar, hidroeléctrica, etc.), agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/generacion/estructura-renovables", añadiendo los parámetros que sean necesarios.

  La documentación de la API la encontrarás en [este link](https://www.ree.es/es/apidatos). Recuerda leer en detenimiento la documentación. 

- **Datos del Instituto Nacional de Estadística (INE):** Además de los datos de la REE, debes extraer y utilizar datos socioeconómicos de las siguientes páginas del INE:

- **Datos Demográficos:** Extraer los datos de población por provincias, diferenciando por grupos de edad, sexo, y extrajeros. Estos datos serán utilizados para analizar cómo la población afecta a la demanda eléctrica en cada provincia.

  - **Página web:** [INE - Población por provincias](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990)

  - "Principales series 1998-2022" --> "Por provincia" --> " Población por provincias, edad (3 grupos de edad), españoles/Extranjeros, Sexo y año"

- **Datos Económicos:**

  - **Página web:** [INE - PIB por provincias](https://www.ine.es/dynt3/inebase/es/index.htm?padre=10426&capsel=10429). 

  - **Pasos para la extracción**:" Resultados provinciales. Serie contable 2016-2021" --> "P.I.B. a precios de mercado y valor añadido bruto a precios básicos por ramas de actividad: Precios corrientes por provincias y periodo."



NOTA1: Tienes que sacar muchos datos, pero recuerda que hemos aprendido herramientas de asincronia que te pueden ayudar en este paso de la ETL. 

NOTA2: Todos estos datos los debes sacar para los años 2019-2020-2021

In [54]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import os

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains, Keys
from time import sleep
# Selenium para establecer la configuración del driver
# -----------------------------------------------------------------------
from selenium import webdriver

# Para generar una barra de proceso en los bucles for
# -----------------------------------------------------------------------
from tqdm import tqdm


In [7]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

meses = {'Enero' : 1,
         'Febrero' : 2,
         'Marzo' : 3,
         'Abril' : 4,
         'Mayo' : 5,
         'Junio' : 6,
         'Julio' : 7,
         'Agosto' : 8,
         'Septiembre' : 9,
         'Octubre' : 10,
         'Noviembre' : 11,
         'Diciembre' : 12
}

In [3]:
headers = {"Accept": 'application/json',
           "Content-Type": 'application/json',
           "Host": 'apidatos.ree.es'}

In [53]:
ruta_datos = r'C:\Users\yanru\OneDrive\Documentos\Hackio\Hackio Bootcamp\Labs\Lab-ELT-Extraccion\datos\demanda'
os.makedirs(ruta_datos, exist_ok= True)
for comun, geo_id in cod_comunidades.items():
    for year in range(2019, 2022):
        ruta_anios = os.path.join(ruta_datos, str(year))
        os.makedirs(ruta_anios, exist_ok= True)
        response = requests.get(f'https://apidatos.ree.es/es/datos/demanda/evolucion?start_date={year}-01-01T00:00&end_date={year}-12-31T23:59&time_trunc=month&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={geo_id}', headers=headers)
        # print(response.status_code)
        result = response.json()
        ruta_com = os.path.join(ruta_anios, comun)
        for i in range(len(result["included"])):
            pd.DataFrame(result["included"][i]["attributes"]["values"]).to_csv(f'{ruta_com}.csv')

In [5]:
pd.DataFrame(result["included"][i]["attributes"]["values"])

,value,percentage,datetime
0,146298.760,1,2021-01-01T00:00:00.000+01:00
1,129503.338,1,2021-02-01T00:00:00.000+01:00
2,144652.408,1,2021-03-01T00:00:00.000+01:00
3,129506.033,1,2021-04-01T00:00:00.000+02:00
4,130651.022,1,2021-05-01T00:00:00.000+02:00
5,128050.949,1,2021-06-01T00:00:00.000+02:00
6,140864.511,1,2021-07-01T00:00:00.000+02:00
7,130850.035,1,2021-08-01T00:00:00.000+02:00
8,133323.629,1,2021-09-01T00:00:00.000+02:00
9,135918.940,1,2021-10-01T00:00:00.000+02:00


In [52]:
ruta_datos = r'C:\Users\yanru\OneDrive\Documentos\Hackio\Hackio Bootcamp\Labs\Lab-ELT-Extraccion\datos\generacion'
os.makedirs(ruta_datos, exist_ok= True)
for comun, geo_id in cod_comunidades.items():
    for year in range(2019, 2022):
        ruta_anios = os.path.join(ruta_datos, str(year))
        os.makedirs(ruta_anios, exist_ok= True)
        response = requests.get(f'https://apidatos.ree.es/es/datos/generacion/estructura-renovables?start_date={year}-01-01T00:00&end_date={year}-12-31T23:59&time_trunc=month&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={geo_id}', headers=headers)
        if response.status_code != 200:
            break
        result = response.json()
        ruta_com = os.path.join(ruta_anios, comun)
        for i in range(len(result["included"])):
            pd.DataFrame(result["included"][i]["attributes"]["values"]).to_csv(f'{ruta_com}.csv')

## Scraping

In [112]:
prefs = {
    "download.default_directory": r'C:\Users\yanru\OneDrive\Documentos\Hackio\Hackio Bootcamp\Labs\Lab-ELT-Extraccion\datos\INE',  # AQUÍ CADA UNO TENDREMOS QUE PONER LA RUTA QUE QUERAMOS PARA QUE SE GUARDEN LOS ARCHIVOS DESCARGADOS
    "download.prompt_for_download": False,   # desactiva el diálogo que Chrome normalmente muestra para pedir confirmación del usuario antes de descargar un archivo
    "directory_upgrade": True,    # hace que Chrome actualice el directorio de descarga predeterminado a la nueva ubicación especificada por download.default_directory si esta ha cambiado.
}

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("prefs", prefs)

url = 'https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990'

driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
sleep(2)
driver.find_element(By.ID, "aceptarCookie").click()
driver.find_element(By.LINK_TEXT, 'Por provincia').click()
sleep(0.5)
driver.find_element('xpath', '/html/body/div[1]/main/div[2]/ul/li[4]/ul/li[1]/a').click()
sleep(1)
driver.find_element('xpath', '//*[@id="tg0"]/div/fieldset/div[2]/button[1]').click()
driver.find_element('xpath', '//*[@id="tg1"]/div/fieldset/div[2]/button[1]').click()
driver.find_element('xpath', '//*[@id="tg2"]/div/fieldset/div[2]/button[1]').click()
driver.find_element('xpath', '//*[@id="tg3"]/div/fieldset/div[2]/button[1]').click()
anios = driver.find_element('xpath', '//*[@id="periodo"]').find_elements(By.TAG_NAME, 'option')
for a in anios:
    if (int(a.text) in range(2019,2022)) or (a.get_attribute('selected') == 'true' and int(a.text) not in range(2019,2022)):
        sleep(0.1)
        a.click()
    else:
        break

driver.find_element('xpath', '//*[@id="botonConsulSele"]').click()
WebDriverWait(driver, 20).until(EC.element_to_be_clickable(('xpath', '//*[@id="btnDescargaForm"]'))).click()
download_frame = driver.find_element('xpath', '//*[@id="thickBoxINEfrm"]')
driver.switch_to.frame(download_frame)
WebDriverWait(driver, 20).until(EC.element_to_be_clickable(('xpath', '//*[@id="export"]/ul/li[4]/label'))).click()
sleep(2)
driver.quit()


In [113]:
ine_dir = r'C:\Users\yanru\OneDrive\Documentos\Hackio\Hackio Bootcamp\Labs\Lab-ELT-Extraccion\datos\INE'
filename = os.listdir(ine_dir)[0]
os.rename(os.path.join(ine_dir, filename), os.path.join(ine_dir, 'datos_demograficos.csv'))

In [114]:
import re
prefs = {
    "download.default_directory": r'C:\Users\yanru\OneDrive\Documentos\Hackio\Hackio Bootcamp\Labs\Lab-ELT-Extraccion\datos\INE',  # AQUÍ CADA UNO TENDREMOS QUE PONER LA RUTA QUE QUERAMOS PARA QUE SE GUARDEN LOS ARCHIVOS DESCARGADOS
    "download.prompt_for_download": False,   # desactiva el diálogo que Chrome normalmente muestra para pedir confirmación del usuario antes de descargar un archivo
    "directory_upgrade": True,    # hace que Chrome actualice el directorio de descarga predeterminado a la nueva ubicación especificada por download.default_directory si esta ha cambiado.
}

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("prefs", prefs)

url = 'https://www.ine.es/dynt3/inebase/es/index.htm?padre=10426&capsel=10429'
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
sleep(1)
driver.find_element(By.ID, "aceptarCookie").click()
driver.find_element('xpath', '/html/body/div[1]/main/div[2]/ul/li[3]/ul/li[1]/a').click()
sleep(1)
driver.find_element('xpath', '//*[@id="tg0"]/div/fieldset/div[2]/button[1]').click()
driver.find_element('xpath', '//*[@id="tg1"]/div/fieldset/div[2]/button[1]').click()

anios = driver.find_element('xpath', '//*[@id="periodo"]').find_elements(By.TAG_NAME, 'option')
for a in anios:
    a_text = re.search(pattern='\d{4}', string = a.text).group()
    if a.get_attribute('selected') == 'true' and int(a_text) in range(2019,2022):
        continue
    elif (int(a_text) in range(2019,2022)):
        sleep(0.1)
        a.click()
driver.find_element('xpath', '//*[@id="botonConsulSele"]').click()
WebDriverWait(driver, 20).until(EC.element_to_be_clickable(('xpath', '//*[@id="btnDescargaForm"]'))).click()
download_frame = driver.find_element('xpath', '//*[@id="thickBoxINEfrm"]')
driver.switch_to.frame(download_frame)
WebDriverWait(driver, 20).until(EC.element_to_be_clickable(('xpath', '//*[@id="export"]/ul/li[4]/label'))).click()
sleep(2)
driver.quit()

In [115]:
ine_dir = r'C:\Users\yanru\OneDrive\Documentos\Hackio\Hackio Bootcamp\Labs\Lab-ELT-Extraccion\datos\INE'
filename = os.listdir(ine_dir)[0]
os.rename(os.path.join(ine_dir, filename), os.path.join(ine_dir, 'datos_pib.csv'))